In [1]:
%reload_ext autoreload
%autoreload 2
#%reload_ext notexbook
#%texify

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import torch

import os

os.chdir('/home/axtr7550/Chromosome_project/cellbgnet/')
import cellbgnet
import cellbgnet.utils

from cellbgnet.datasets import DataSimulator
from cellbgnet.utils.hardware import cpu, gpu
from cellbgnet.model import CellBGModel
from cellbgnet.simulation.psf_kernel import SMAPSplineCoefficient
from cellbgnet.generic.emitter import EmitterSet
from cellbgnet.train_loss_infer import generate_probmap_cells

os.chdir('/home/axtr7550/Chromosome_project/cellbgnet/notebooks')

from skimage.io import imread
from skimage.measure import label
import random
import edt
from skimage.filters import gaussian
from scipy.ndimage import rotate
import random
import pickle
import pathlib
from pathlib import Path
%matplotlib qt5

In [10]:
param_file = '../cellbgnet/utils/reference_files/reference.yaml'
param = cellbgnet.utils.param_io.ParamHandling().load_params(param_file)

In [11]:
psf_params = param.PSF.to_dict()
simulation_params = param.Simulation.to_dict()
hardware_params = param.Hardware.to_dict()
train_size = simulation_params['train_size']

In [12]:
model = cellbgnet.model.CellBGModel(param)

INITIATING CUDA IMPLEMENTATION
training sliding windows on camera chip:
Area num: 0, field_xy: [0, 127, 0, 127]
Area num: 1, field_xy: [114, 241, 0, 127]
Area num: 2, field_xy: [228, 355, 0, 127]
Area num: 3, field_xy: [342, 469, 0, 127]
Area num: 4, field_xy: [456, 583, 0, 127]
Area num: 5, field_xy: [570, 697, 0, 127]
Area num: 6, field_xy: [684, 811, 0, 127]
Area num: 7, field_xy: [798, 925, 0, 127]
Area num: 8, field_xy: [912, 1039, 0, 127]
Area num: 9, field_xy: [1026, 1153, 0, 127]
Area num: 10, field_xy: [1140, 1267, 0, 127]
Area num: 11, field_xy: [1174, 1301, 0, 127]
Area num: 12, field_xy: [0, 127, 114, 241]
Area num: 13, field_xy: [114, 241, 114, 241]
Area num: 14, field_xy: [228, 355, 114, 241]
Area num: 15, field_xy: [342, 469, 114, 241]
Area num: 16, field_xy: [456, 583, 114, 241]
Area num: 17, field_xy: [570, 697, 114, 241]
Area num: 18, field_xy: [684, 811, 114, 241]
Area num: 19, field_xy: [798, 925, 114, 241]
Area num: 20, field_xy: [912, 1039, 114, 241]
Area num: 21,

In [13]:
model.init_recorder()

In [18]:
model.init_eval_data()

Eval image generation:   3%|█▏                                 | 1/30 [00:00<00:05,  5.15it/s]

IN CUDA! /nIN CUDA! /n

Eval image generation:  10%|███▌                               | 3/30 [00:00<00:04,  5.83it/s]

IN CUDA! /nIN CUDA! /n

Eval image generation:  17%|█████▊                             | 5/30 [00:00<00:04,  5.97it/s]

...

IN CUDA! /n

Eval image generation:  97%|████████████████████████████████▊ | 29/30 [00:05<00:00,  4.92it/s]

IN CUDA! /nIN CUDA! /n

Eval image generation: 100%|██████████████████████████████████| 30/30 [00:05<00:00,  5.32it/s]


eval images shape: (30, 1041, 1302) contain 10184 molecules,


#### Looking at training data

In [19]:
model.training(train_size, simulation_params)

IN CUDA! /n

(tensor(255.3123, device='cuda:0'),
 (tensor(68.4397, device='cuda:0'),
  tensor(75.5228, device='cuda:0'),
  tensor(1.8551, device='cuda:0'),
  tensor(70.7640, device='cuda:0')))

In [16]:

density = simulation_params['density']
random_filename = random.choice(model.data_generator.cell_mask_filenames)
random_cell_mask = imread(random_filename)
random_cell_mask = segmentation.expand_labels(random_cell_mask, distance=1)
print('sampling random filename', random_filename)
non_cell_density = simulation_params['non_cell_density']

# generate probabilty map and cell mask crop from this one random cell mask
prob_map, cell_masks_batch = generate_probmap_cells(random_cell_mask, model.batch_size, 
                                    train_size, density, non_cell_density,
                                    simulation_params['margin_empty']) 

type: name 'segmentation' is not defined

In [9]:
prob_map.shape, cell_masks_batch.shape

((16, 128, 128), (16, 128, 128))

In [10]:
index = 9
fig, ax = plt.subplots(nrows=1, ncols=2)
ax[0].imshow(prob_map[index])
ax[0].set_title('Prob map')
ax[1].imshow(cell_masks_batch[index])
ax[1].set_title('Cell masks')
plt.show()

In [11]:
imgs_sim, xyzi_gt, s_mask, psf_imgs_gt, locs, field_xy = model.data_generator.simulate_data( 
    prob_map=gpu(prob_map), batch_size=model.batch_size,
    local_context=model.local_context,
    photon_filter=model.train_params['photon_filter'],
    photon_filter_threshold=model.train_params['photon_filter_threshold'],
    P_locs_cse=model.train_params['P_locs_cse'],
    iter_num=model._iter_count, train_size=train_size,
    robust_training=model.data_generator.simulation_params['robust_training'],
    cell_masks=cell_masks_batch
)


In [12]:
field_xy

tensor([  0, 127,   0, 127])

In [13]:
index = 2
plt.figure()
plt.imshow(imgs_sim[index, 0].cpu().numpy())
plt.show()
plt.figure()
plt.imshow(locs[index].cpu().numpy())
plt.show()

### Adding field coordinates to train loop test